<a href="https://colab.research.google.com/github/anujavenkatachalam04/chvi_vbd_rj/blob/main/notebooks/ncd_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NCD Preprocessing

In [ ]:
import os
import pandas as pd
import geopandas as gpd
import requests
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
drive.mount('/content/drive')
import geopandas as gpd
from shapely.geometry import Point
import uuid
import re
import datetime

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
os.chdir("/content/drive/MyDrive/CHVI")

In [11]:
ncd_raw="1_Data/NCD/raw"

In [124]:
blocks_shp=gpd.read_file("5_Shapefiles/SUBDISTRICT_11/Rajasthan_Blocks.geojson")

In [15]:
diab=[file for file in os.listdir(f"{ncd_raw}/diabetes") if file.endswith(".csv")]

In [17]:
# consolidate_diab
diab_all=pd.DataFrame()
for file in tqdm(diab):
  diab_all=pd.concat([diab_all,pd.read_csv(f"{ncd_raw}/diabetes/{file}")])

100%|██████████| 21/21 [00:16<00:00,  1.31it/s]


In [19]:
# consolidate_ht
ht=[file for file in os.listdir(f"{ncd_raw}/hypert") if file.endswith(".csv")]
ht_all=pd.DataFrame()
for file in tqdm(ht):
  ht_all=pd.concat([ht_all,pd.read_csv(f"{ncd_raw}/hypert/{file}")])

100%|██████████| 21/21 [00:14<00:00,  1.48it/s]


In [21]:
diab_all.columns, ht_all,

Index(['country_id', 'country', 'state_id', 'state_name', 'district_id',
       'district_name', 'taluka_id', 'taluka_name', 'population',
       'target_population', 'enrolled_all', 'enrolled_30', 'screened',
       'rescreened', 'referred_by_screening', 'referred_for_via_screening',
       'diagnosed', 'under_treatment', 'examined', 'via_examinations',
       'comorbidity', 'control', 'followup_adherence', 'Month', 'Year'],
      dtype='object')

In [32]:
rename_cols_list=['enrolled_all', 'enrolled_30', 'screened',
       'rescreened', 'referred_by_screening', 'referred_for_via_screening',
       'diagnosed', 'under_treatment', 'examined', 'via_examinations',
       'comorbidity', 'control', 'followup_adherence']

In [30]:
def rename_cols(disease, cols_to_rename, cols):
    return [f"{disease}_{col}" if col in cols_to_rename else col for col in cols]

In [34]:
ht_all.columns=rename_cols("ht", rename_cols_list, list(ht_all.columns))

In [35]:
diab_all.columns=rename_cols("diab", rename_cols_list, list(diab_all.columns))

In [36]:
ht_all.columns, diab_all.columns

(Index(['country_id', 'country', 'state_id', 'state_name', 'district_id',
        'district_name', 'taluka_id', 'taluka_name', 'population',
        'target_population', 'ht_enrolled_all', 'ht_enrolled_30', 'ht_screened',
        'ht_rescreened', 'ht_referred_by_screening',
        'ht_referred_for_via_screening', 'ht_diagnosed', 'ht_under_treatment',
        'ht_examined', 'ht_via_examinations', 'ht_comorbidity', 'ht_control',
        'ht_followup_adherence', 'Month', 'Year'],
       dtype='object'),
 Index(['country_id', 'country', 'state_id', 'state_name', 'district_id',
        'district_name', 'taluka_id', 'taluka_name', 'population',
        'target_population', 'diab_enrolled_all', 'diab_enrolled_30',
        'diab_screened', 'diab_rescreened', 'diab_referred_by_screening',
        'diab_referred_for_via_screening', 'diab_diagnosed',
        'diab_under_treatment', 'diab_examined', 'diab_via_examinations',
        'diab_comorbidity', 'diab_control', 'diab_followup_adherence', 'M

In [43]:
len(ht_all), len(diab_all)

(8589, 8589)

In [231]:
ncd=ht_all.merge(diab_all, how="outer")

In [232]:
ncd=ncd[['district_id',
       'district_name', 'taluka_id', 'taluka_name', 'Month', 'Year','ht_diagnosed','diab_diagnosed', 'diab_enrolled_all','ht_enrolled_all']]

# Mapping NCD to Census 2011 using CHIP data (lat-long of AWC is used to find the block)


In [270]:
ncd_chip_map=pd.read_excel(f"{ncd_raw}/mapping_files/district_taluka_ncd.xlsx")

In [271]:
ncd_chip_map=ncd_chip_map[["taluka_id","chip_block_id"]]

In [272]:
ncd_chip_map.isna().sum()

,0
taluka_id,0
chip_block_id,11


In [273]:
len(ncd), len(ncd_chip_map)

(8589, 485)

In [274]:
ncd_mapped=ncd.merge(ncd_chip_map, how="left")

In [275]:
# blocks missing chip block id are invalid
ncd_mapped[(ncd_mapped["chip_block_id"].isna())]["taluka_name"].unique()

array(['K', 'C', 'Delet block', 'Wrong Data', 'UU DELETE', 'Z delet',
       'blank 2', 'wrong block'], dtype=object)

In [276]:
ncd_mapped=ncd_mapped.dropna(subset="chip_block_id")

In [277]:
ncd_mapped

,district_id,district_name,taluka_id,taluka_name,Month,Year,ht_diagnosed,diab_diagnosed,diab_enrolled_all,ht_enrolled_all,chip_block_id
0,10801,Ganganagar,108001,Karanpur,April,2024,3199.0,2587.0,127355,127355,126.0
1,10801,Ganganagar,108001,Karanpur,April,2025,6817.0,5212.0,142845,142845,126.0
2,10801,Ganganagar,108001,Karanpur,August,2024,5138.0,4104.0,133544,133544,126.0
3,10801,Ganganagar,108001,Karanpur,August,2025,7268.0,5469.0,143083,143083,126.0
4,10801,Ganganagar,108001,Karanpur,December,2024,6010.0,4690.0,142363,142363,126.0
...,...,...,...,...,...,...,...,...,...,...,...
9949,10848,Salumber,108482,semari,May,2025,1638.0,409.0,57875,57875,496.0
9950,10848,Salumber,108482,semari,November,2024,1113.0,258.0,30305,30305,496.0
9951,10848,Salumber,108482,semari,October,2024,938.0,234.0,28996,28996,496.0
9952,10848,Salumber,108482,semari,September,2024,752.0,197.0,27659,27659,496.0


In [278]:
# map chip_block_id to lat, long using awc data

In [345]:
# get awc lat-long
awc_data=pd.read_csv(f"{ncd_raw}/mapping_files/awc_lat_long.csv")

In [346]:
# get mapping of awc to block id
awc_block_id=pd.read_csv(f"{ncd_raw}/mapping_files/chip_awc_block.csv")

In [347]:
awc_block_id=awc_block_id[["block_id","anganwadi_id"]]

In [348]:
awc_block_id=awc_block_id.drop_duplicates(subset=["block_id","anganwadi_id"])

In [349]:
len(awc_block_id), len(awc_data)

(65127, 58994)

In [350]:
awc_data=awc_data.merge(awc_block_id, how="outer")

In [351]:
# 12% awcs are missing lat, longs
awc_data.isna().sum()/len(awc_data)

,0
anganwadi_id,0.000000
median_lat,0.123356
median_lng,0.123356
block_id,0.020867


In [352]:
# Compute centroid per block_id using mean lat/lon
block_centroids = (
    awc_data.dropna(subset=["median_lat", "median_lng"], how="any").groupby(by="block_id")[["median_lat", "median_lng"]]
    .mean()
    .reset_index()
    .rename(columns={"median_lat": "centroid_lat", "median_lng": "centroid_lng"})
)

print(block_centroids.head())


   block_id  centroid_lat  centroid_lng
0       1.0     26.498218     74.737770
1       2.0     26.355787     75.037162
2       3.0     26.022645     74.780578
3       4.0     25.967486     74.228917
4       5.0     25.964676     75.173050


In [353]:
block_centroids.isna().sum()

,0
block_id,0
centroid_lat,0
centroid_lng,0


In [354]:
# save chip block centroids
block_centroids.to_csv(f"{ncd_raw}/mapping_files/chip_block_centroids.csv", index=False)

In [357]:
# merge block centroids to ncd data
ncd_mapped_chip=ncd_mapped.merge(block_centroids, left_on="chip_block_id", right_on="block_id", how="left").drop(columns="block_id")

In [358]:
len(ncd_mapped_chip)

9744

In [364]:
# check missing centroids - i.w., where no AWCs were surveyed inthe block/no awc lat-longs in the entire block
ncd_mapped_chip[ncd_mapped_chip["centroid_lat"].isna()][["chip_block_id", "district_name", "taluka_id","taluka_name"]].drop_duplicates()

,chip_block_id,district_name,taluka_id,taluka_name
2751,473.0,Jaipur,108327,SIRSI
7308,202.0,Kota,108195,Khairabad


In [365]:
# fix these manually by looking up the centroid from google maps - by tehsil polygon
ncd_mapped_chip.loc[ncd_mapped_chip["taluka_id"]==108327, "centroid_lat"]=26.91072
ncd_mapped_chip.loc[ncd_mapped_chip["taluka_id"]==108327, "centroid_lng"]=75.68197

In [367]:
# fix these manually by looking up the centroid from google maps - by tehsil polygon
ncd_mapped_chip.loc[ncd_mapped_chip["taluka_id"]==108195, "centroid_lat"]=24.65458
ncd_mapped_chip.loc[ncd_mapped_chip["taluka_id"]==108195, "centroid_lng"]=75.92918

In [368]:
# check missing centroids - i.w., where no AWCs were surveyed inthe block/no awc lat-longs in the entire block
ncd_mapped_chip[ncd_mapped_chip["centroid_lat"].isna()][["chip_block_id", "district_name", "taluka_id","taluka_name"]].drop_duplicates()

,chip_block_id,district_name,taluka_id,taluka_name


In [371]:
ncd_mapped_chip


,district_id,district_name,taluka_id,taluka_name,Month,Year,ht_diagnosed,diab_diagnosed,diab_enrolled_all,ht_enrolled_all,chip_block_id,centroid_lat,centroid_lng
0,10801,Ganganagar,108001,Karanpur,April,2024,3199.0,2587.0,127355,127355,126.0,29.852564,73.514929
1,10801,Ganganagar,108001,Karanpur,April,2025,6817.0,5212.0,142845,142845,126.0,29.852564,73.514929
2,10801,Ganganagar,108001,Karanpur,August,2024,5138.0,4104.0,133544,133544,126.0,29.852564,73.514929
3,10801,Ganganagar,108001,Karanpur,August,2025,7268.0,5469.0,143083,143083,126.0,29.852564,73.514929
4,10801,Ganganagar,108001,Karanpur,December,2024,6010.0,4690.0,142363,142363,126.0,29.852564,73.514929
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9739,10848,Salumber,108482,semari,May,2025,1638.0,409.0,57875,57875,496.0,24.033681,73.857062
9740,10848,Salumber,108482,semari,November,2024,1113.0,258.0,30305,30305,496.0,24.033681,73.857062
9741,10848,Salumber,108482,semari,October,2024,938.0,234.0,28996,28996,496.0,24.033681,73.857062
9742,10848,Salumber,108482,semari,September,2024,752.0,197.0,27659,27659,496.0,24.033681,73.857062


In [373]:
# Map to census
ncd_gdf = gpd.GeoDataFrame(ncd_mapped_chip,
    geometry=gpd.points_from_xy(ncd_mapped_chip["centroid_lng"], ncd_mapped_chip["centroid_lat"]),
    crs="EPSG:4326"  # WGS84 lat-long
)

In [374]:
ncd_joined = gpd.sjoin(
    ncd_gdf,
    blocks_shp[["NAME", "DISTRICT", "C_CODE11", "geometry"]],
    how="left",
    predicate="within"
)

In [380]:
ncd_joined=ncd_joined[['NAME', 'DISTRICT', 'C_CODE11', 'Month',
       'Year', 'ht_diagnosed', 'diab_diagnosed', 'diab_enrolled_all',
       'ht_enrolled_all', 'chip_block_id']]

In [383]:
ncd_joined["ht_prop"]=ncd_joined["ht_diagnosed"]/ncd_joined["ht_enrolled_all"]
ncd_joined["diab_prop"]=ncd_joined["diab_diagnosed"]/ncd_joined["diab_enrolled_all"]

/tmp/ipython-input-275024888.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ncd_joined["ht_prop"]=ncd_joined["ht_diagnosed"]/ncd_joined["ht_enrolled_all"]
/tmp/ipython-input-275024888.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ncd_joined["diab_prop"]=ncd_joined["diab_diagnosed"]/ncd_joined["diab_enrolled_all"]


In [397]:
# Combine Year and Month into a single datetime column, then convert to period
ncd_joined["Year_Month"] = pd.to_datetime(
    ncd_joined["Year"].astype(str) + "-" + ncd_joined["Month"].astype(str) + "-01"
).dt.to_period("M")


/tmp/ipython-input-3014834984.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ncd_joined["Year_Month"] = pd.to_datetime(


In [398]:
ncd_joined.drop(columns=["Date", "Month", "Year"], inplace=True)

/tmp/ipython-input-1448233603.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ncd_joined.drop(columns=["Date", "Month", "Year"], inplace=True)


In [400]:
ncd_joined.columns

Index(['NAME', 'DISTRICT', 'C_CODE11', 'ht_diagnosed', 'diab_diagnosed',
       'diab_enrolled_all', 'ht_enrolled_all', 'chip_block_id', 'ht_prop',
       'diab_prop', 'Year_Month'],
      dtype='object')

In [405]:
ncd_joined.to_csv("1_Data/NCD/clean/ncd_data_2024_2025.csv", index=False)

In [ ]:
# The End!